<a href="https://colab.research.google.com/github/anshhikaa/anshhikaa/blob/main/recommendationEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle

with open('saved_files/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('saved_files/cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)


In [4]:
!mkdir internship_app
%cd internship_app

# create main files
!touch app.py
!touch recommender.py
!mkdir data


mkdir: cannot create directory ‘internship_app’: File exists
/content/internship_app
mkdir: cannot create directory ‘data’: File exists


In [ ]:
# copy saved cleaned file into the project folder
!cp /content/cleaned_internships.csv /content/internship_app/data/


In [5]:
%%writefile recommender.py
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("data/cleaned_internships.csv")

df["combined"] = (
    df["Job Title"].fillna("") + " " +
    df["Job Type"].fillna("") + " " +
    df["Duration"].fillna("")
)

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df["combined"])

def recommend_internships(user_query, top_n=5):
    user_vec = vectorizer.transform([user_query])
    similarities = cosine_similarity(user_vec, tfidf_matrix).flatten()

    top_indices = similarities.argsort()[::-1][:top_n]

    return df.iloc[top_indices][["Job Title", "Company Name", "Stipend", "Duration", "Cities"]]


Overwriting recommender.py


In [14]:
%%writefile app.py
import streamlit as st
import pandas as pd
from recommender import recommend_internships

st.set_page_config(page_title="Internship Recommender", layout="wide")

st.title("🎓 Internship Recommendation System")

st.write("Enter your skills / interests to get internship suggestions.")

# Input box
user_query = st.text_input("Your Skills or Desired Internship Role:")

# When button is clicked → show results
if st.button("Get Recommendations"):
    if user_query.strip() == "":
        st.warning("Please enter something.")
    else:
        results = recommend_internships(user_query)
        st.success("Top Recommendations:")

        st.dataframe(results)



Overwriting app.py


In [7]:
import pandas as pd

df = pd.read_csv("internship companies.csv")
df.head()



,Unnamed: 0,company,internship,location,start date,duration,stipend,posted on,apply by
0,0,Internshala,Web Development,Gurgaon,Immediately,6 Months,20000 /month,29 Feb'20,18 Apr'20
1,1,Delhi Technological University - Karyon,Campus Ambassador,Work From Home,Immediately,1 Month,Performance Based,18 Feb'20,19 Apr'20
2,2,Internshala,Operations,Gurgaon,15 Apr'20,6 Months,20000 /month,18 Mar'20,29 Apr'20
3,3,Your Digital Boat,Content Writing,Work From Home,Immediately,2 Months,5000 /month,10 Apr'20,8 May'20
4,4,American Institute Of Big Data Professionals,Web Development,Work From Home,Immediately,1 Month,2000-5000 /month,10 Apr'20,8 May'20


In [65]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.head()


,unnamed:_0,company,internship,location,start_date,duration,stipend,posted_on,apply_by,combined_text
0,0,Internshala,Web Development,Gurgaon,Immediately,6 Months,20000 /month,29 Feb'20,18 Apr'20,20000 /month 6 Months
1,1,Delhi Technological University - Karyon,Campus Ambassador,Work From Home,Immediately,1 Month,Performance Based,18 Feb'20,19 Apr'20,Performance Based 1 Month
2,2,Internshala,Operations,Gurgaon,15 Apr'20,6 Months,20000 /month,18 Mar'20,29 Apr'20,20000 /month 6 Months
3,3,Your Digital Boat,Content Writing,Work From Home,Immediately,2 Months,5000 /month,10 Apr'20,8 May'20,5000 /month 2 Months
4,4,American Institute Of Big Data Professionals,Web Development,Work From Home,Immediately,1 Month,2000-5000 /month,10 Apr'20,8 May'20,2000-5000 /month 1 Month


In [28]:
import os

# create folder if not exists
os.makedirs("/content/internship_app/data", exist_ok=True)

# save cleaned CSV
df.to_csv("/content/internship_app/data/cleaned_internships.csv", index=False)

print("Saved at /content/internship_app/data/cleaned_internships.csv")



Saved at /content/internship_app/data/cleaned_internships.csv


In [30]:
!ls /content/internship_app/data


cleaned_internships.csv


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
df = pd.read_csv("/content/internship_app/data/cleaned_internships.csv")

# Combine useful text fields into one "combined_text"
text_columns = []

# detect which columns exist
for col in ['job_title', 'company_name', 'job_type', 'stipend', 'duration', 'cities', 'states']:
    if col in df.columns:
        text_columns.append(col)

df['combined_text'] = df[text_columns].fillna('').agg(' '.join, axis=1)

# TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

def recommend_internships(user_query, n=5):
    """
    user_query: text (skills, keywords, job interests)
    """
    user_tfidf = vectorizer.transform([user_query])
    similarity = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

    # top N recommendations
    idx = similarity.argsort()[-n:][::-1]

    # return those rows
    return df.iloc[idx][['job_title', 'company_name', 'stipend', 'duration', 'cities', 'states']]


In [5]:
import os
os.makedirs("/content/internship_app/data", exist_ok=True)
print("Folder created!")


Folder created!


In [33]:
import pandas as pd

df = pd.read_csv("internship companies.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.to_csv("/content/internship_app/data/cleaned_internships.csv", index=False)

print("Saved cleaned dataset to internship_app/data/")


Saved cleaned dataset to internship_app/data/


In [10]:
recommender_code = """
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/content/internship_app/data/cleaned_internships.csv')

text_columns = [col for col in ['job_title','company_name','job_type','stipend','duration','cities','states'] if col in df.columns]
df['combined_text'] = df[text_columns].fillna('').agg(' '.join, axis=1)

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

def recommend_internships(user_query, n=5):
    user_tfidf = vectorizer.transform([user_query])
    similarity = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    idx = similarity.argsort()[-n:][::-1]
    return df.iloc[idx][['job_title','company_name','stipend','duration','cities','states']]
"""

with open("/content/internship_app/recommender.py", "w") as f:
    f.write(recommender_code)

print("recommender.py created!")


recommender.py created!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [55]:
import pandas as pd

df = pd.read_csv("/content/internship_app/data/cleaned_internships.csv")
df.head()
df.columns


Index(['unnamed:_0', 'company', 'internship', 'location', 'start_date',
       'duration', 'stipend', 'posted_on', 'apply_by'],
      dtype='object')

In [12]:
%%writefile /content/internship_app/recommender.py
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/content/internship_app/data/cleaned_internships.csv')

# Combine meaningful text fields
text_columns = ['company', 'internship', 'location', 'duration', 'stipend']

df['combined_text'] = df[text_columns].fillna('').agg(' '.join, axis=1)

# Create TF-IDF vectors
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

def recommend_internships(user_query, n=5):
    user_tfidf = vectorizer.transform([user_query])
    similarity = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    top_idx = similarity.argsort()[-n:][::-1]

    # FIX: return all columns required by app.py
    return df.iloc[top_idx][[
        'internship',
        'company',
        'location',
        'start_date',
        'duration',
        'stipend',
        'posted_on',
        'apply_by'
    ]]


Overwriting /content/internship_app/recommender.py


In [13]:
import os
os.chdir("/content/internship_app")

from recommender import recommend_internships
recommend_internships("python data analysis machine learning", 5)


KeyError: "['job_title', 'company_name', 'cities', 'states'] not in index"

In [13]:
!pip install streamlit
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!chmod +x ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.7 MB/s eta 0:00:00
--2025-11-28 16:40:45--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 75.2.60.68, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  17.2MB/s    in 0.8s    

2025-11-28 16:40:47 (17.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [36]:
%cd /content/internship_app
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 >/content/log.txt 2>&1 &


/content/internship_app


In [54]:
!./ngrok config add-authtoken 35wHRSzxv1Vitty8Cqp9QtRMk9w_6jJWDkPApUC8R9PE8EfFS


NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [40]:
%cd /content/internship_app
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 > log.txt 2>&1 &


/content/internship_app


In [43]:
%cd /content/internship_app
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 > log.txt 2>&1 &


/content/internship_app


In [48]:
!pip install pyngrok


In [46]:
%cd /content/internship_app
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 > log.txt 2>&1 &


/content/internship_app


In [68]:
!wget -O ngrok.zip https://dashboard.ngrok.com/api/tunnels/download/linux
!unzip -o ngrok.zip
!chmod +x ngrok


--2025-11-28 17:09:19--  https://dashboard.ngrok.com/api/tunnels/download/linux
Resolving dashboard.ngrok.com (dashboard.ngrok.com)... 54.193.106.71, 184.169.255.245, 13.57.90.134, ...
Connecting to dashboard.ngrok.com (dashboard.ngrok.com)|54.193.106.71|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-11-28 17:09:19 ERROR 404: Not Found.

Archive:  ngrok.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ngrok.zip or
        ngrok.zip.zip, and cannot find ngrok.zip.ZIP, period.


In [1]:
%cd /content/internship_app
!./ngrok config add-authtoken 35wHRSzxv1Vitty8Cqp9QtRMk9w_6jJWDkPApUC8R9PE8EfFS


/content/internship_app
NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the c

In [6]:
!curl -I localhost:8501


HTTP/1.1 200 OK
Server: TornadoServer/6.5.1
Content-Type: text/html
Date: Fri, 28 Nov 2025 17:59:18 GMT
Accept-Ranges: bytes
Etag: "90673d71dae2c59955303c5cba4b4db3c5495e7812df8dee0ccd00963b88ec584a8ebfcc1e22f4767e9afb8939904251da4e83be5ff541ed00e7df975e1ff143"
Last-Modified: Fri, 28 Nov 2025 16:40:42 GMT
Cache-Control: no-cache
Content-Length: 1522
Vary: Accept-Encoding



In [2]:
!wget -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip
!chmod +x ngrok



--2025-11-28 18:40:58--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11047287 (11M) [application/octet-stream]
Saving to: ‘ngrok.zip’

ngrok.zip           100%[===================>]  10.54M  14.5MB/s    in 0.7s    

2025-11-28 18:40:59 (14.5 MB/s) - ‘ngrok.zip’ saved [11047287/11047287]

Archive:  ngrok.zip
  inflating: ngrok                   


In [6]:
!./ngrok config add-authtoken 35wHRSzxv1Vitty8Cqp9QtRMk9w_6jJWDkPApUC8R9PE8EfFS


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
!nohup streamlit run /content/internship_app/app.py --server.port 8501 --server.address 0.0.0.0 > log.txt 2>&1 &


In [16]:
!./ngrok http 8501 --log=stdout


INFO[11-28|19:02:12] no configuration paths supplied 
WARN[11-28|19:02:12] ngrok config file found at both XDG and legacy locations, using XDG location xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml
INFO[11-28|19:02:12] using configuration at default config path path=/root/.config/ngrok/ngrok.yml
INFO[11-28|19:02:12] open config file                         path=/root/.config/ngrok/ngrok.yml err=nil
t=2025-11-28T19:02:12+0000 lvl=info msg="FIPS 140 mode" enabled=false
t=2025-11-28T19:02:12+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2025-11-28T19:02:12+0000 lvl=info msg="client session established" obj=tunnels.session
t=2025-11-28T19:02:12+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2025-11-28T19:02:12+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:8501 url=https://thyroarytenoid-platonically-shelley.ngrok-free.dev
t=2025-11-28T19:02:20+0000 lvl=info msg="j